### Chat with the model cards for some questions to ask

#### step0. set up the environment and the dependencies

In [1]:
import sys
sys.path.append("..")
import warnings
warnings.filterwarnings("ignore")
import os

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

#### step1. load the model cards

In [2]:
from src.io import load_cards
data_root = "./data/"

model_cards = load_cards(data_root, "model")
print(f"There are {len(model_cards)} model repo cards")

Loading the model cards from ./data/model_cards.json
There are 1300 model repo cards


In [9]:
from src.io import load_json

ft_models = load_json(os.path.join(data_root, "ft_models.json"))
pt_models = load_json(os.path.join(data_root, "pt_models.json"))
failed_cards = load_json(os.path.join(data_root, "chat_failed_model_cards.json"))

len(ft_models), len(pt_models), len(failed_cards)

(321, 783, 0)

In [3]:
from src.utils import show_cards

show_cards(model_cards, type="model", num=2, sample="head")


========================= The model card for the repo address: https://huggingface.co/microsoft/phi-2 =========================

Model Summary
Phi-2 is a Transformer with 2.7 billion parameters. It was trained using the same data sources as Phi-1.5, augmented with a new data source that consists of various NLP synthetic texts and filtered websites (for safety and educational value). When assessed against benchmarks testing common sense, language understanding, and logical reasoning, Phi-2 showcased a nearly state-of-the-art performance among models with less than 13 billion parameters.
Our model hasn't been fine-tuned through reinforcement learning from human feedback. The intention behind crafting this open-source model is to provide the research community with a non-restricted small model to explore vital safety challenges, such as reducing toxicity, understanding societal biases, enhancing controllability, and more.
Intended Uses
Given the nature of the training data, the Phi-2 mod

#### step2. chat with the model cards for finding the fine-tuned models

In [4]:
query_cards = {k: v for k,v in model_cards.items() if k not in ft_models and k not in pt_models and k not in failed_cards}
len(query_cards)

1104

In [5]:
from src.chat import chat_cards

query_with_explanation = "Is this model fine-tuned for some specific downstream tasks, \
like classificiation, summarization, translation, code generation, math-problem solving?\n\
REMEMBER: the tasks like conversational chat, instruction following do NOT count\n\
REMEMBER: the response should start with 'YES' or 'NO', and then follow a very brief explanation"

responses, failed_cards = chat_cards(
    query=query_with_explanation,
    cards={k: query_cards[k] for i, k in enumerate(query_cards) if i < 3}, # query_cards[:3]
    type="model",
    llm="gpt-3.5-turbo-1106",
    mode="each",
    verbose=True, # with the response printed out for each card
    process_bar='none'
)


========================= Chatting with the model from its card with the repo: runwayml/stable-diffusion-inpainting =========================

NO, this model is not fine-tuned for specific downstream tasks like classification, summarization, translation, code generation, or math-problem solving. It is primarily designed for text-to-image generation and inpainting.

========================= Chatting with the model from its card with the repo: Salesforce/blip-image-captioning-base =========================

YES, this model is fine-tuned for specific downstream tasks such as image-text retrieval, image captioning, and VQA (Visual Question Answering).

========================= Chatting with the model from its card with the repo: tiiuae/falcon-7b =========================

NO, Falcon-7B is not fine-tuned for specific downstream tasks like classification, summarization, translation, code generation, or math-problem solving.


In [6]:
from src.chat import chat_cards

query_binary = "Is this model fine-tuned for some specific downstream tasks, \
like classificiation, summarization, translation, code generation, math-problem solving?\n\
REMEMBER: the tasks like conversational chat, instruction following do NOT count\n\
REMEMBER: the response should only contain 'YES' or 'NO', without any explanation"

responses, failed_cards = chat_cards(
    query=query_binary,
    cards=query_cards,
    type="model",
    llm="gpt-3.5-turbo-1106",
    mode="each",
    verbose=False, # quietly
    process_bar='notebook'
)

  0%|          | 0/1104 [00:00<?, ?it/s]

In [7]:
previous_cnt = len(ft_models)
for resp in responses:
    if resp['response'].startswith('YES'):
        ft_models.append(resp['repo_addr'])
    else:
        pt_models.append(resp['repo_addr'])

print(f"There are {len(ft_models)-previous_cnt} / {len(responses)} models that are fine-tuned for some specific downstream tasks:")

There are 321 / 1104 models that are fine-tuned for some specific downstream tasks:


In [13]:
from src.io import save_json

save_json(ft_models, os.path.join(data_root, "ft_models.json"))
save_json(pt_models, os.path.join(data_root, "pt_models.json"))
save_json(failed_cards, os.path.join(data_root, "chat_failed_model_cards.json"))